In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/sbert_medical_model/model")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/sbert_medical_model/tokenizer")
y_encoder = joblib.load("/content/drive/MyDrive/sbert_medical_model/label_encoder.pkl")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [ ]:
import joblib

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import joblib
from google.colab import drive

drive.mount('/content/drive')

model_path = '/content/drive/MyDrive/sbert_medical_model'
model = AutoModelForSequenceClassification.from_pretrained(f"{model_path}/model")
tokenizer = AutoTokenizer.from_pretrained(f"{model_path}/tokenizer")
y_encoder = joblib.load(f"{model_path}/label_encoder.pkl")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
def predict_specialty(text):
    inputs = tokenizer(
        text,
        truncation=True,
        max_length=256,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        pred_label = torch.argmax(logits, dim=1).cpu().item()

    return y_encoder.inverse_transform([pred_label])[0]

In [ ]:
input_text = "Severe heartache."
predicted_specialty = predict_specialty(input_text)
print(f"Input: {input_text}\nPredicted Specialty: {predicted_specialty}\n")

Input: Severe heartache.
Predicted Specialty: Cardiology



In [ ]:
input_text = "depression"
predicted_specialty = predict_specialty(input_text)
print(f"Input: {input_text}\nPredicted Specialty: {predicted_specialty}\n")

Input: depression
Predicted Specialty: Psychology/Psychiatry



In [ ]:
input_text = "Constant sneezing"
predicted_specialty = predict_specialty(input_text)
print(f"Input: {input_text}\nPredicted Specialty: {predicted_specialty}\n")

Input: Constant sneezing
Predicted Specialty: ENT and Pulmonology

